In [1]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import os
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [10]:
df = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',
                 sep=';',
                 parse_dates=['MESS_DATUM','MESS_DATUM_WOZ'],
                 date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d%H:%M"))
display(df.head, df.dtypes)

/var/folders/c5/491yhz3s1y93gvqdkz7t62p00000gn/T/ipykernel_14770/3133747922.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('/Users/tomkurze/Stackfuel DPP /Stackfuel-DPP/data/raw/Balkonkraftwerk/produkt_st_stunde_20050101_20251231_01975.txt',


<bound method NDFrame.head of         STATIONS_ID          MESS_DATUM  QN_592  ATMO_LBERG  FD_LBERG  \
0              1975 2005-01-01 00:23:00       1       120.0       0.0   
1              1975 2005-01-01 01:23:00       1       100.0       0.0   
2              1975 2005-01-01 02:23:00       1        89.0       0.0   
3              1975 2005-01-01 03:23:00       1       104.0       0.0   
4              1975 2005-01-01 04:23:00       1       111.0       0.0   
...             ...                 ...     ...         ...       ...   
184026         1975 2025-12-31 19:23:00       1       116.0       0.0   
184027         1975 2025-12-31 20:23:00       1       121.0       0.0   
184028         1975 2025-12-31 21:23:00       1       124.0       0.0   
184029         1975 2025-12-31 22:23:00       1       123.0       0.0   
184030         1975 2025-12-31 23:23:00       1       121.0       0.0   

        FG_LBERG  SD_LBERG   ZENIT      MESS_DATUM_WOZ  eor  
0            0.0         0  148

STATIONS_ID                int64
MESS_DATUM        datetime64[ns]
QN_592                     int64
ATMO_LBERG               float64
FD_LBERG                 float64
FG_LBERG                 float64
SD_LBERG                   int64
ZENIT                    float64
MESS_DATUM_WOZ    datetime64[ns]
eor                       object
dtype: object

In [4]:
# Abfrage der Daten auf entso-e
# Ausfühle Informationen unter https://documenter.getpostman.com/view/7009892/2s93JtP3F6#3b383df0-ada2-49fe-9a50-98b1bb201c6b

energy_prices_url = 'https://web-api.tp.entsoe.eu/api?'

response = requests.get(url=energy_prices_url,
                        params=
                        {'securityToken': os.getenv('TOKEN'), # Das muss ich noch anpassen, dass es sicherer ist
                         'DocumentType': 'A44',
                         'periodStart': '202501010000',
                         'periodEnd': '202601010000',
                         'out_Domain': '10Y1001A1001A82H',
                         'in_Domain': '10Y1001A1001A82H',
                         'contract_MarketAgreement.type': 'A01'
                        })

# Antwort verarbeiten
if response.status_code == 200:
    # Direkte Nutzung der XML-Daten (ohne Dekomprimierung)
    xml_data = response.content.decode('utf-8')  # Bytes zu String dekodieren

    # XML-Daten parsen
    root = ET.fromstring(xml_data)

    # Namespace für die XML-Daten (falls vorhanden)
    namespaces = {
        'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'
    }

    # Liste für die Daten erstellen
    data = []

    # Relevante Daten extrahieren (A44-Dokumenttyp)
    for timeseries in root.findall('.//ns:TimeSeries', namespaces):
        mrid = timeseries.find('ns:mRID', namespaces).text
        for period in timeseries.findall('.//ns:Period', namespaces):
            start = period.find('ns:timeInterval/ns:start', namespaces).text
            end = period.find('ns:timeInterval/ns:end', namespaces).text
            resolution = period.find('ns:resolution', namespaces).text
            for point in period.findall('.//ns:Point', namespaces):
                position = point.find('ns:position', namespaces).text
                price = point.find('ns:price.amount', namespaces).text
                data.append({
                    'mRID': mrid,
                    'Startzeit': start,
                    'Endzeit': end,
                    'Resolution': resolution,
                    'Position': position,
                    'Preis': float(price)  # Preis als Float speichern
                })

    # DataFrame erstellen
    df = pd.DataFrame(data)

    # DataFrame anzeigen
    print(df.head())

    # Optional: DataFrame in eine CSV-Datei speichern
    #df.to_csv('energy_prices_2025_2026.csv', index=False)
    #print("Daten wurden erfolgreich in 'energy_prices_2025_2026.csv' gespeichert!")

else:
    print(f"Fehler bei der Anfrage: {response.status_code}")
    print(response.text)

  mRID          Startzeit            Endzeit Resolution Position  Preis
0    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4    1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54


In [37]:
print(df)

      mRID          Startzeit            Endzeit Resolution Position  Preis
0        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        1  22.40
1        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        2  -7.40
2        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        3 -10.16
3        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        4  -5.30
4        1  2024-12-31T23:00Z  2025-01-01T23:00Z      PT15M        5   5.54
...    ...                ...                ...        ...      ...    ...
50039  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       90   9.61
50040  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       91   4.50
50041  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       92   0.10
50042  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       93   1.14
50043  732  2025-12-31T23:00Z  2026-01-01T23:00Z      PT15M       94   0.00

[50044 rows x 6 columns]
